# This notebook runs the Deep Gravity model on the yearly trade prediction dataset

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import sys
import tqdm

import random
import torch.utils.data.distributed
import torch
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

import model_utils
from data_compiler import FlowDataset
from deepgravity import DeepGravity

sys.path.append('../GeoDS_mobility_predictions/')
import parameters

# random seeds
torch.manual_seed(parameters.seed)
np.random.seed(parameters.seed)
random.seed(parameters.seed)

torch_device = torch.device("cpu")

In [2]:
##############
# Load data
##############

nodes = pd.read_csv(parameters.node_path)
if parameters.domain=='Google':
    node_targets = pd.read_csv(parameters.node_target_path)
    nodes = pd.merge(nodes, node_targets, on=[parameters.node_id], how='inner')

nodes_columns = parameters.node_features + parameters.node_targets + [parameters.node_id] + [parameters.node_timestamp]
nodes_columns = [i for i in nodes_columns if i != ""]
nodes_columns = nodes_columns
nodes = nodes[nodes_columns]

if parameters.domain=='Google':
    # Rename and melt
    nodes.rename(columns={parameters.node_id: parameters.flow_origin}, inplace=True)

    nodes = pd.melt(nodes, id_vars=parameters.node_features+[parameters.flow_origin]+[parameters.node_timestamp],
                            value_vars=parameters.node_targets, var_name='destination', value_name='Value')

edges = pd.read_csv(parameters.edge_path)
if parameters.domain=="GeoDS":
    edge_targets = pd.read_csv(parameters.edge_target_path)
    edges = pd.merge(edges, edge_targets, on=[parameters.flow_origin, parameters.flow_destination], how='inner')

edges_columns = parameters.flows_features + [parameters.flow_origin] + \
    [parameters.flow_destination] + [parameters.flows_timestamp] + \
        [parameters.flows_value]
edges_columns = [i for i in edges_columns if i != ""]
edges = edges[edges_columns]

if parameters.domain=='Google':
    unit = [parameters.flow_origin, parameters.node_timestamp]
    target_value = 'Value'
else:
    unit = [parameters.flow_origin, parameters.flows_timestamp]
    target_value = parameters.flows_value

##############
# Initial cleaning
##############

#nodes = nodes.fillna(0)
#edges = edges.fillna(0)

In [4]:
nodes.isna().any()

population_2019              False
population_density_2019      False
residential_land_use_area    False
commercial_land_use_area     False
industrial_land_use_area     False
retail_land_use_area         False
natural_land_use_area        False
residential_roads_length     False
other_roads_length           False
main_roads_length            False
point_transport              False
building_transport           False
point_food                   False
building_food                False
point_health                 False
building_health              False
point_education              False
building_education           False
point_retail                 False
building_retail              False
iso_3166_2_code              False
dtype: bool

In [3]:
##############
# Create data objects
##############

columns = {'node_id': parameters.node_id,
           'node_timestamp': parameters.node_timestamp,
           'flow_origin': parameters.flow_origin,
           'flow_destination': parameters.flow_destination,
           'flows_timestamp': parameters.flows_timestamp,
           'target_value':target_value}

flow_data = FlowDataset(domain=parameters.domain,
                        columns=columns,
                        unit = unit,
                        nodes=nodes,
                        edges=edges,)

In [4]:
flow_data.data_dict[('US-AL', 0)]

population_2019  population_density_2019  \
origin Timeline                                             
US-AL  0                 4903185                96.811652   
       0                 4903185                96.811652   
       0                 4903185                96.811652   
       0                 4903185                96.811652   
       0                 4903185                96.811652   
       0                 4903185                96.811652   

                 residential_land_use_area  commercial_land_use_area  \
origin Timeline                                                        
US-AL  0                          6.049055                  0.273233   
       0                          6.049055                  0.273233   
       0                          6.049055                  0.273233   
       0                          6.049055                  0.273233   
       0                          6.049055                  0.273233   
       0                          6.049055                  0.273233   

                 industrial_land_use_area  retail_land_use_area  \
origin Timeline                                                   
US-AL  0                          1.76191              0.165395   
       0                          1.76191              0.165395   
       0                          1.76191              0.165395   
       0                          1.76191              0.165395   
       0                          1.76191              0.165395   
       0                          1.76191              0.165395   

                 natural_land_use_area  residential_roads_length  \
origin Timeline                                                    
US-AL  0                    528.038165              1.252222e+08   
       0                    528.038165              1.252222e+08   
       0                    528.038165              1.252222e+08   
       0                    528.038165              1.252222e+08   
       0                    528.038165              1.252222e+08   
       0                    528.038165              1.252222e+08   

                 other_roads_length  main_roads_length  ...  point_food  \
origin Timeline                                         ...               
US-AL  0               1.112820e+08       1.112820e+08  ...        2566   
       0               1.112820e+08       1.112820e+08  ...        2566   
       0               1.112820e+08       1.112820e+08  ...        2566   
       0               1.112820e+08       1.112820e+08  ...        2566   
       0               1.112820e+08       1.112820e+08  ...        2566   
       0               1.112820e+08       1.112820e+08  ...        2566   

                 building_food  point_health  building_health  \
origin Timeline                                                 
US-AL  0                    11           361               21   
       0                    11           361               21   
       0                    11           361               21   
       0                    11           361               21   
       0                    11           361               21   
       0                    11           361               21   

                 point_education  building_education  point_retail  \
origin Timeline                                                      
US-AL  0                    1766                 170           957   
       0                    1766                 170           957   
       0                    1766                 170           957   
       0                    1766                 170           957   
       0                    1766                 170           957   
       0                    1766                 170           957   

                 building_retail  \
origin Timeline                    
US-AL  0                       7   
       0                       7   
       0                       7   
      

In [5]:
# Create a list of FlowDataset objects
flow_data_chunked = flow_data.create_chunks(chunk_size=parameters.chunk_size, window_size=parameters.window_size)

# Add past values to each chunk
[flow_chunk.add_past_values(periods=parameters.lag_periods,
                            edge_columns = parameters.time_dependent_edge_columns,
                            node_columns = parameters.time_dependent_node_columns) for flow_chunk in tqdm.tqdm(flow_data_chunked)]

# Add target to each chunk
[flow_chunk.add_target_values() for flow_chunk in tqdm.tqdm(flow_data_chunked)]

# Create a list of FlowDataset objects
train_data_chunked = []
validation_data_chunked = []
test_data_chunked = []

for flow_data in tqdm.tqdm(flow_data_chunked):
    train_data, validation_data, test_data = flow_data.split_train_validate_test(validation_period = parameters.validation_period)
    train_data_chunked.append(train_data)
    validation_data_chunked.append(validation_data)
    test_data_chunked.append(test_data)

100%|██████████| 13/13 [00:01<00:00, 11.10it/s]


In [14]:
validation_data_chunked[0].data_dict[('US-AL', 348)]

,population_2019,population_density_2019,residential_land_use_area,commercial_land_use_area,industrial_land_use_area,retail_land_use_area,natural_land_use_area,residential_roads_length,other_roads_length,main_roads_length,...,point_health,building_health,point_education,building_education,point_retail,building_retail,destination,Value,Value_1,Value_target
0,4903185,96.811652,6.049055,0.273233,1.76191,0.165395,528.038165,1.252222e+08,1.112820e+08,1.112820e+08,...,361,21,1766,170,957,7,retail_and_recreation_percent_change_from_base...,-18.0,-15.0,-16.0
1,4903185,96.811652,6.049055,0.273233,1.76191,0.165395,528.038165,1.252222e+08,1.112820e+08,1.112820e+08,...,361,21,1766,170,957,7,grocery_and_pharmacy_percent_change_from_baseline,-9.0,-9.0,-9.0
2,4903185,96.811652,6.049055,0.273233,1.76191,0.165395,528.038165,1.252222e+08,1.112820e+08,1.112820e+08,...,361,21,1766,170,957,7,parks_percent_change_from_baseline,-21.0,-15.0,-12.0
3,4903185,96.811652,6.049055,0.273233,1.76191,0.165395,528.038165,1.252222e+08,1.112820e+08,1.112820e+08,...,361,21,1766,170,957,7,transit_stations_percent_change_from_baseline,-16.0,-15.0,-17.0
4,4903185,96.811652,6.049055,0.273233,1.76191,0.165395,528.038165,1.252222e+08,1.112820e+08,1.112820e+08,...,361,21,1766,170,957,7,workplaces_percent_change_from_baseline,-22.0,-23.0,-22.0
5,4903185,96.811652,6.049055,0.273233,1.76191,0.165395,528.038165,1.252222e+08,1.112820e+08,1.112820e+08,...,361,21,1766,170,957,7,residential_percent_change_from_baseline,8.0,8.0,8.0


In [17]:
parameters.flow_destination

'destination'

In [4]:
prediction_list = []
for chunk in range(len(train_data_chunked[:1])):
    # Set config where parameters are tuned
    config = {
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16]),
        "dim_hidden": tune.sample_from(lambda _: 2**np.random.randint(2, 6)),
        "dropout_p": tune.choice([0.25, 0.35, 0.45]),
        "num_layers": tune.choice([5, 10, 15]),
    }
    # Set scheduler
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=parameters.epochs,
        grace_period=1,
        reduction_factor=2)
    # Set reporter
    reporter = CLIReporter(
            # parameter_columns=["lr", "batch_size", "dim_hidden", "dropout_p", "num_layers"],
            metric_columns=["loss", "training_iteration"])
    # Run tuning
    result = tune.run(
        tune.with_parameters(model_utils.train_and_validate_deepgravity, train_data_chunked = train_data_chunked,
                validation_data_chunked = validation_data_chunked, chunk = chunk, momentum = parameters.momentum,
                epochs = parameters.epochs),
        resources_per_trial={"cpu": 4},
        config=config,
        num_samples=10,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))

    input_dim = train_data_chunked[chunk].get_feature_dim()
    best_trained_model = DeepGravity(dim_input = input_dim,
                                    dim_hidden = best_trial.config["dim_hidden"],
                                    dropout_p = best_trial.config["dropout_p"],
                                    num_layers = best_trial.config["num_layers"],)

    best_checkpoint = result.get_best_checkpoint(trial=best_trial, metric="loss", mode="min")
    best_checkpoint_dir = best_checkpoint.to_directory(path=os.path.join(parameters.output_path, "best_checkpoints", "trade", str(chunk)))
    model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, f"checkpoint_{str(datetime.datetime.now()).replace(' ', '_')[:19]}"))
    best_trained_model.load_state_dict(model_state)

    test_data_loader = torch.utils.data.DataLoader(test_data_chunked[chunk], batch_size=4)
    model_utils.test(test_data_loader, best_trained_model, test_data_chunked[chunk], loss_fn = None, store_predictions=True)
    print("Finished prediction on test set")
    prediction_list.append(test_data_chunked[chunk].compile_predictions(columns_to_rename = parameters.columns_to_rename))
#pd.concat(prediction_list, axis=0).to_csv(f"{parameters.output_path}/prediction_{str(datetime.datetime.now()).replace(' ', '_')[:19]}.csv")

== Status ==
Current time: 2023-04-08 18:23:35 (running for 00:00:00.11)
Memory usage on this node: 5.7/8.0 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 4.0/4 CPUs, 0/0 GPUs, 0.0/2.65 GiB heap, 0.0/1.33 GiB objects
Result logdir: /Users/rgyuri/ray_results/train_and_validate_deepgravity_2023-04-08_18-23-35
Number of trials: 2/2 (1 PENDING, 1 RUNNING)
+--------------------------------------------+----------+-----------------+--------------+--------------+-------------+-----------+--------------+
| Trial name                                 | status   | loc             |   batch_size |   dim_hidden |   dropout_p |        lr |   num_layers |
|--------------------------------------------+----------+-----------------+--------------+--------------+-------------+-----------+--------------|
| train_and_validate_deepgravity_b5d43_00000 | RUNNING  | 127.0.0.1:22294 |            8 |           16 |  

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_and_validate_deepgravity_b5d43_00000,2023-04-08_18-24-04,True,,e50e7a39776d43dbb1d430181d3cb72d,"0_batch_size=8,dim_hidden=16,dropout_p=0.4500,lr=0.0749,num_layers=5",Gyorgys-MacBook-Pro.local,2,1.31703e+17,127.0.0.1,22294,True,14.0685,3.17818,14.0685,1680971044,0,,2,b5d43_00000,0.0116789
train_and_validate_deepgravity_b5d43_00001,2023-04-08_18-24-19,True,,e50e7a39776d43dbb1d430181d3cb72d,"1_batch_size=4,dim_hidden=32,dropout_p=0.4500,lr=0.0112,num_layers=15",Gyorgys-MacBook-Pro.local,2,4.32091e+13,127.0.0.1,22294,True,14.8015,0.916771,14.8015,1680971059,0,,2,b5d43_00001,0.0116789


(train_and_validate_deepgravity pid=22294) Finished training!
== Status ==
Current time: 2023-04-08 18:24:09 (running for 00:00:34.81)
Memory usage on this node: 6.2/8.0 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: -1.3170295056000614e+17 | Iter 1.000: -4.0227602027554703e+18
Resources requested: 4.0/4 CPUs, 0/0 GPUs, 0.0/2.65 GiB heap, 0.0/1.33 GiB objects
Result logdir: /Users/rgyuri/ray_results/train_and_validate_deepgravity_2023-04-08_18-23-35
Number of trials: 2/2 (1 RUNNING, 1 TERMINATED)
+--------------------------------------------+------------+-----------------+--------------+--------------+-------------+-----------+--------------+-------------+----------------------+
| Trial name                                 | status     | loc             |   batch_size |   dim_hidden |   dropout_p |        lr |   num_layers |        loss |   training_iteration |
|--------------------------------------------+------------+--------------

2023-04-08 18:24:19,700	INFO tune.py:798 -- Total run time: 44.61 seconds (44.57 seconds for the tuning loop).


== Status ==
Current time: 2023-04-08 18:24:19 (running for 00:00:44.59)
Memory usage on this node: 5.6/8.0 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: -6.587307983691627e+16 | Iter 1.000: -2.0114376190430828e+18
Resources requested: 0/4 CPUs, 0/0 GPUs, 0.0/2.65 GiB heap, 0.0/1.33 GiB objects
Result logdir: /Users/rgyuri/ray_results/train_and_validate_deepgravity_2023-04-08_18-23-35
Number of trials: 2/2 (2 TERMINATED)
+--------------------------------------------+------------+-----------------+--------------+--------------+-------------+-----------+--------------+-------------+----------------------+
| Trial name                                 | status     | loc             |   batch_size |   dim_hidden |   dropout_p |        lr |   num_layers |        loss |   training_iteration |
|--------------------------------------------+------------+-----------------+--------------+--------------+-------------+-----------+--------------+-

In [ ]:
#set([i[1] for i in train_data_chunked[0].data_dict.keys()])
#train_data_chunked[0].data_dict[(32, 1996)].columns
#data_loader = torch.utils.data.DataLoader(flow_data_chunked[0], batch_size=parameters.batch_size)
#for X, y in data_loader:
#    print(f"Shape of X : {X.shape}")
#    print(f"Shape of y: {y.shape} {y.dtype}")
#    break